In [9]:
import pandas as pd
import os
import tqdm

In [10]:
locales = ['/cool_temp_maize_soybean/', '/warm_temp_maize_soybean_irrigated/', '/warm_temp_maize-soybean_dryland/']
path = locales[0]
files = ['soil_c','surf_water','flux_soc','soil_water','p_flux','temp', 'n_flux',
         'plant_c','plant_n','plant_p','canopcy_c','plant_stress','photosynthesis','plant_growth']
#files.append('soil_temp', 'canopy_temp')
start_year = 2015
growth_period = 8

In [11]:
def JoinCsvsByYear(start_year: int, period: int, file_name : str, path: str, external_save: bool) -> pd.DataFrame:
    to_aggr = []
    for i in range(period):
        #if (start_year == 2001 and (file_name == 'soil_temp' or file_name == 'canopy_temp')):
        #    continue
            
        df = pd.read_csv(path + str(start_year) + file_name + '.csv')
        #some files have hourly data, so turn them into daily data
        #do yourself a favor and optimize this monstrository at some point pls
        if ('HOUR' in df.columns):
            df = df.drop(['DATE' , 'HOUR'], axis=1)
            df['DATE'] = pd.date_range(start='1/1/' + str(start_year), periods=len(df), freq='H')
            df.set_index('DATE', inplace=True)
            df = df.resample('D').mean().reset_index()
            
        #index used to make join faster 
        df.set_index('DATE')
        to_aggr.append(df)
        start_year += 1
    merged_data = pd.concat(to_aggr).reset_index().drop('index', axis=1)    
    if (external_save):
        abs_path = os.path.join(path, 'with_plant_soil_details/')
        if not os.path.exists(abs_path):
            os.makedirs(abs_path)
        merged_data.to_csv(os.path.join(abs_path, file_name + '.csv'))

        return merged_data

In [12]:
for file in tqdm.tqdm(files):
    df = JoinCsvsByYear(start_year, growth_period, file, '../datasets' + path + 'csv_outs/', True)  

100%|███████████████████████████████████████████| 14/14 [00:03<00:00,  4.53it/s]
